### Projects Meta Data

In [5]:
# the file downloaded from planet websites https://app.plant-for-the-planet.org/app/projects?_scope=map&currency=EUR&tenant=ten_NxJq55pm&filter[purpose]=trees,conservation&locale=en
import pandas as pd
import json
import requests
import ast
from shapely.geometry import shape, Polygon, MultiPolygon,mapping
from shapely.ops import transform
import pyproj
import numpy as np
from scipy import stats
from shapely import wkt
import geopandas as gpd
df=pd.read_csv("C:/Users/Angela/Forest_Monitoring/input/Plant_for_planet_shapefiles/plantforplanet_projects.csv")

In [6]:

projects_dict = df.groupby('properties.id').apply(lambda x: x.drop('properties.id', axis=1).to_dict(orient='records')).to_dict()

projects_dict

C:\Users\Angela\AppData\Local\Temp\ipykernel_11348\3776630632.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  projects_dict = df.groupby('properties.id').apply(lambda x: x.drop('properties.id', axis=1).to_dict(orient='records')).to_dict()


{'proj_08KPr5UC3XliXwWgDqNgiIHs': [{'Unnamed: 0': 65,
   'type': 'Feature',
   'geometry.type': 'Point',
   'geometry.coordinates': '[40.739686581322246, -2.317548419577491]',
   'properties._scope': 'map',
   'properties.allowDonations': True,
   'properties.classification': 'large-scale-planting',
   'properties.countPlanted': 813710.0,
   'properties.countTarget': 1800000.0,
   'properties.country': 'KE',
   'properties.currency': 'EUR',
   'properties.fixedRates': '[]',
   'properties.image': '631bb1e028d86881019932.jpg',
   'properties.isApproved': True,
   'properties.isFeatured': False,
   'properties.isPublished': True,
   'properties.isTopProject': True,
   'properties.location': 'Kenya',
   'properties.minTreeCount': 2.0,
   'properties.name': 'Eden Reforestation Projects - Kenya',
   'properties.paymentDefaults.fixedTreeCountOptions': '[10, 25, 50, 100]',
   'properties.paymentDefaults.fixedDefaultTreeCount': 5.0,
   'properties.purpose': 'trees',
   'properties.reviewScore'

In [7]:


# Saving theretrived information to  Json file

all_projects = df['properties.id'].unique()


project_info = {}
for slug in all_projects:
    url = f"https://app.plant-for-the-planet.org/app/projects/{slug}?_scope=extended&currency=EUR&locale=en"
    response = requests.get(url)
    if response.status_code == 200:
        project_info[slug] = response.json()
    else:
        project_info[slug] = f"Failed to retrieve information for {slug}"

with open('project_info.json', 'w') as json_file:
    json.dump(project_info, json_file)

print("Project information saved to '../input/Plant_for_planet_shapefiles/project_info.json'")

Project information saved to 'project_info.json'


In [10]:
# Reading the all the data in the json file and we transpose to have the right format data
json_file_path = 'C:/Users/Angela/Forest_Monitoring/input/Plant_for_planet_shapefiles/project_info.json'
df = pd.read_json(json_file_path)
df.head()
df = df.T
df.head()

,id,_scope,allowDonations,certificates,classification,coordinates,countDonated,countPlanted,countRegistered,countTarget,...,treeCost,unitCost,unitType,unitsContributed,unitsTargeted,videoUrl,visitorAssistance,website,yearAbandoned,yearAcquired
proj_ezpAp1POh20dBnYpx0BjhU35,proj_ezpAp1POh20dBnYpx0BjhU35,extended,True,[],agroforestry,"{'lon': -49.5903, 'lat': -9.2045}",313,313,0,2000,...,3.9,3.9,tree,{'tree': 313.0},{'tree': 2000},https://www.youtube.com/watch?v=9GrWYgJnFqM,True,https://ecosystemrestorationcamps.org/camp-far...,2012,2012
proj_ZCspL8JYmUu0OXcx6O73I1j0,proj_ZCspL8JYmUu0OXcx6O73I1j0,extended,True,[],other-planting,"{'lon': -43.468833, 'lat': -22.495032}",3403,3404,0,1000000,...,5.03,5.03,tree,{'tree': 3415.5},{'tree': 1000000},None,True,http://www.itpa.org.br,2012,None
proj_nXBzA2sbX2tm1D75p7bfJ81Z,proj_nXBzA2sbX2tm1D75p7bfJ81Z,extended,True,[],large-scale-planting,"{'lon': -1.979117, 'lat': 8.2204420012607}",41321,41321,0,10000000,...,2.0,2.0,tree,{'tree': 43027.0},{'tree': 10000000},https://youtu.be/f7koI_yKNtI?si=Wtu9dCV8mcCMZxq2,True,www.plant-for-the-planet.org,1996,2020
proj_wMNVTLkRIZ4TcV8oi26wlRwN,proj_wMNVTLkRIZ4TcV8oi26wlRwN,extended,True,[],agroforestry,"{'lon': 104.791856, 'lat': 20.700794}",149,149,0,100000,...,8.5,8.5,tree,{'tree': 149.0},{'tree': 100000},None,True,trofaco.org,2018,2021
proj_vH52lGTvcjbCGKG4dINKsXJU,proj_vH52lGTvcjbCGKG4dINKsXJU,extended,False,[],agroforestry,"{'lon': -67.945206, 'lat': 6.169667}",24,30024,30000,212500,...,13.17,13.17,tree,{'tree': 24},{'tree': 212500},None,False,https://www.kardianuts.com/,0,None


In [13]:
df["description"][0]

C:\Users\Angela\AppData\Local\Temp\ipykernel_11348\2162132602.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["description"][0]


'This project will be implemented at Farm of the Future, a member of the Ecosystem Restoration Communities movement. \nThrough the establishment of agroforestry systems, this project will prove to local communities’ members and local authorities that there are eco-friendly and economically viable alternatives to the monoculture plantation model. \nFirst, a pilot agroforestry system will be planted at Farm of the Future. About 2,000 trees will be planted on a plot of 1ha. This area will function as a demonstration site. \nAfter the establishment of this demonstration site, at least three local smallholder farmers will establish agroforestry systems on their land and become in turn examples for the broader community. \nThis project has also an important educational goal: each tree planted will directly finance an agroforestry training program that will educate and empower at least 120 high school students per year and will ensure the long-term sustainability of the project. \nThe ‘Rural 

In [14]:
# Resetting the index without keeping the old index as a column
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 61 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     263 non-null    object
 1   _scope                 263 non-null    object
 2   allowDonations         263 non-null    object
 3   certificates           263 non-null    object
 4   classification         257 non-null    object
 5   coordinates            263 non-null    object
 6   countDonated           257 non-null    object
 7   countPlanted           257 non-null    object
 8   countRegistered        257 non-null    object
 9   countTarget            263 non-null    object
 10  country                263 non-null    object
 11  currency               263 non-null    object
 12  degradationCause       171 non-null    object
 13  degradationYear        249 non-null    object
 14  description            263 non-null    object
 15  employeesCount         

In [15]:

columns_to_keep = ['id', 'countPlanted', 'country',"description","firstTreePlanted","sites","survivalRate"]

# Filtering only the colunms we are interested  in as the data has 64 columns some of which are not useful in our analysis
df= df[columns_to_keep]


df.head()

,id,countPlanted,country,description,firstTreePlanted,sites,survivalRate
0,proj_ezpAp1POh20dBnYpx0BjhU35,313,BR,This project will be implemented at Farm of th...,None,[{'geometry': {'coordinates': [[[-49.958833158...,80
1,proj_ZCspL8JYmUu0OXcx6O73I1j0,3404,BR,"ITPA was born in 1998, from the initiative of ...",2012-11-18 00:00:00,"[{'geometry': {'coordinates': [[[-43.472501, -...",90
2,proj_nXBzA2sbX2tm1D75p7bfJ81Z,41321,GH,Plant-for-Ghana is a hybrid restoration agrofo...,2021-07-24 00:00:00,"[{'geometry': {'coordinates': [[[-2.01902187, ...",93
3,proj_wMNVTLkRIZ4TcV8oi26wlRwN,149,VN,The site was a forest before. It was cut down ...,2021-08-19 00:00:00,[{'geometry': {'coordinates': [[[104.768595725...,83
4,proj_vH52lGTvcjbCGKG4dINKsXJU,30024,CO,KardiaNuts was established in 2018 through the...,None,[{'geometry': {'coordinates': [[[-67.927780151...,95


In [17]:
# saving the data as csv
df.to_csv('C:/Users/Angela/Forest_Monitoring/input/Plant_for_planet_shapefiles/new_plant_planet.csv', index=False)

# Data Loading

In [78]:

df = pd.read_csv("C:/Users/Angela/Forest_Monitoring/input/Plant_for_planet_shapefiles/new_plant_planet.csv", encoding='latin-1')
df.rename(columns={'Unnamed: 0': 'properties.id'}, inplace=True)

#Checking projects with no geometry information from original data column "sites" cotains each projects sites geometry coordinates
df_with_site_polygon = df[df['sites'] != '[]'].copy()


def remove_not_geom(site):
    try:
        site_data = ast.literal_eval(site)
        if isinstance(site_data, list) and all("geometry" in item for item in site_data):
            return site
        else:
            return None
    except:
        return None

df_with_site_polygon.loc[:, 'sites'] = df_with_site_polygon['sites'].apply(remove_not_geom)
df_with_site_polygon = df_with_site_polygon.dropna(subset=["sites"])

# defining a function to count the total number of polygons as each row of our data has project information which in other cases are multi-polygons
def count_all_polygons(all_geom_data):
    try:
        all_geom_data = ast.literal_eval(all_geom_data)
        count_polygons = 0
        polygon_areas = []
        polygon_geometries = []

        project = pyproj.Transformer.from_crs(
            pyproj.CRS('EPSG:4326'),
            pyproj.CRS('EPSG:6933'), # This to project the data to a plane for easy and accurate area calculation,later we will change back the crs to 4326 before final saving 
            always_xy=True).transform

        for geom_data in all_geom_data:
            if geom_data is None or "geometry" not in geom_data:
                continue
            geometry = shape(geom_data["geometry"])
            if isinstance(geometry, Polygon):
                count_polygons += 1
                projected_polygon = transform(project, geometry)
                # calculating polygon area in metres square
                polygon_areas.append(projected_polygon.area)
                polygon_geometries.append(geometry)
            elif isinstance(geometry, MultiPolygon):
                count_polygons += len(geometry.geoms)
                for polygon in geometry.geoms:
                    projected_polygon = transform(project, polygon)
                    polygon_areas.append(projected_polygon.area)
                    polygon_geometries.append(polygon)

        return count_polygons, polygon_areas, polygon_geometries
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None


df_with_site_polygon[['polygon_count', 'polygon_areas', 'polygons']] = df_with_site_polygon.apply(
    lambda row: pd.Series(count_all_polygons(row['sites'])), axis=1)
# Converting the area to kmsquare

df_with_site_polygon['polygon_areas_km'] = df_with_site_polygon['polygon_areas'].apply(
    lambda x: [area / 1_000_000 for area in x] if x is not None else None)

#flattening a area list  of lists into a single list
flat_polygon_areas = [area for sublist in df_with_site_polygon['polygon_areas'].dropna().tolist() for area in sublist]


df_with_site_polygon['has_negative_area'] = df_with_site_polygon['polygon_areas_km'].apply(
    lambda areas: any(area < 0 for area in areas) if areas is not None else False
)


df_with_positive_areas = df_with_site_polygon[df_with_site_polygon['has_negative_area'] == False].copy()


df_with_positive_areas.drop(columns=['has_negative_area'], inplace=True)


flat_polygon_areas_cleaned = [area for sublist in df_with_positive_areas['polygon_areas'].dropna().tolist() for area in sublist]

if flat_polygon_areas_cleaned:
    max_area = max(flat_polygon_areas_cleaned)
    min_area = min(flat_polygon_areas_cleaned)
else:
    max_area = None
    min_area = None

print(f"Maximum Area: {max_area}")
print(f"Minimum Area: {min_area}")


Error: 'NoneType' object has no attribute 'get'
Error: 'NoneType' object has no attribute 'get'
Error: 'NoneType' object has no attribute 'get'
Error: 'NoneType' object has no attribute 'get'
Error: 'NoneType' object has no attribute 'get'
Maximum Area: 35563594818.56289
Minimum Area: 0.7183456395418495


In [79]:
df_with_positive_areas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253 entries, 0 to 262
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                253 non-null    object 
 1   countPlanted      247 non-null    float64
 2   country           251 non-null    object 
 3   description       253 non-null    object 
 4   firstTreePlanted  132 non-null    object 
 5   sites             253 non-null    object 
 6   survivalRate      156 non-null    float64
 7   polygon_count     248 non-null    float64
 8   polygon_areas     248 non-null    object 
 9   polygons          248 non-null    object 
 10  polygon_areas_km  248 non-null    object 
dtypes: float64(3), object(8)
memory usage: 23.7+ KB


In [80]:

df_with_positive_areas['polygon_count'] .sum()

2406.0

In [81]:
# Rename the dataframe
df_with_site_polygon = df_with_positive_areas

In [82]:
df_with_site_polygon.sort_values(by='polygon_areas', ascending=False)

,id,countPlanted,country,description,firstTreePlanted,sites,survivalRate,polygon_count,polygon_areas,polygons,polygon_areas_km
143,proj_iS4tktbXYmiwI2vq2iqWbViX,3.0,MW,A Tiny Paradise: A Fruit Tree Garden and safe ...,NaN,[{'geometry': {'coordinates': [[[33.4289195356...,100.0,1.0,[35563594818.56289],[POLYGON ((33.42891953566266 -9.72935041815760...,[35563.59481856289]
220,proj_bOT1rwSf5o3vo3WYDvzAbRwe,NaN,ID,The Leuser Ecosystem is the last place on Eart...,NaN,[{'geometry': {'coordinates': [[[97.6904296875...,NaN,1.0,[15346126655.811012],"[POLYGON ((97.6904296875 2.569938616535896, 96...",[15346.126655811013]
102,proj_IayG16fWq0B5ukJE0M7fiiKn,1455050.0,ID,Our goal on Borneo is the large-scale reforest...,NaN,[{'geometry': {'coordinates': [[[113.064422607...,80.0,1.0,[5104354843.621789],[POLYGON ((113.06442260742188 -1.7122389825336...,[5104.354843621789]
80,proj_itaSbQGmpsGPhXkBl9l7iJWk,800150.0,TZ,This Assisted Natural Regeneration (ANR) proje...,NaN,[{'geometry': {'coordinates': [[[30.9673624209...,NaN,1.0,[4757686217.024443],[POLYGON Z ((30.96736242090414 -6.346060065577...,[4757.686217024443]
45,proj_XjyrwvTHqZJ9vC9cquEPdXwQ,22248.0,BR,Idesamâs Carbon Neutral Program (PCN) was cr...,NaN,[{'geometry': {'coordinates': [[[-58.128662109...,80.0,1.0,[3975166028.0934606],[POLYGON ((-58.128662109375 -2.358647500909131...,[3975.1660280934607]
...,...,...,...,...,...,...,...,...,...,...,...
29,proj_mHS0F02qEFlJZULq7MMji6Ke,611867.0,NaN,Die Mutterbaumschule von BÃ¤ume fÃ¼r Menschen ...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",90.0,NaN,None,None,None
48,proj_Cw1ZAjHXvjlC5Dfel2pWf4GX,3413.0,IT,AYUDH Europe is an international youth movemen...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",90.0,NaN,None,None,None
188,proj_kcFeNrstIRmgN9FhHkEIfctV,1888983.0,ZM,The Luanshya district is Zambia's most densely...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",90.0,NaN,None,None,None
193,proj_DbGBDHbePvTCUgndUcvrJodr,6058.0,CI,The region of the TANDA located in the EAST zo...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",89.0,NaN,None,None,None


In [83]:
df_filtered=df_with_site_polygon

### Aggregating Total area in metreSq

In [84]:
df_filtered['total_area'] = df_filtered['polygon_areas'].apply(lambda areas: sum(area for area in areas) if areas is not None else 0)

In [85]:
df_filtered.head()

,id,countPlanted,country,description,firstTreePlanted,sites,survivalRate,polygon_count,polygon_areas,polygons,polygon_areas_km,total_area
0,proj_ezpAp1POh20dBnYpx0BjhU35,313.0,BR,This project will be implemented at Farm of th...,NaN,[{'geometry': {'coordinates': [[[-49.958833158...,80.0,1.0,[13148.354394101978],[POLYGON ((-49.958833158016205 -9.351073341239...,[0.013148354394101978],1.314835e+04
1,proj_ZCspL8JYmUu0OXcx6O73I1j0,3404.0,BR,"ITPA was born in 1998, from the initiative of ...",2012-11-18 00:00:00,"[{'geometry': {'coordinates': [[[-43.472501, -...",90.0,3.0,"[536402.3599409966, 2614147.3272324847, 259015...","[POLYGON Z ((-43.472501 -22.489448 0, -43.4723...","[0.5364023599409966, 2.6141473272324847, 0.259...",3.409565e+06
2,proj_nXBzA2sbX2tm1D75p7bfJ81Z,41321.0,GH,Plant-for-Ghana is a hybrid restoration agrofo...,2021-07-24 00:00:00,"[{'geometry': {'coordinates': [[[-2.01902187, ...",93.0,1.0,[10097443.289694324],"[POLYGON Z ((-2.01902187 8.21742549 0, -2.0202...",[10.097443289694324],1.009744e+07
3,proj_wMNVTLkRIZ4TcV8oi26wlRwN,149.0,VN,The site was a forest before. It was cut down ...,2021-08-19 00:00:00,[{'geometry': {'coordinates': [[[104.768595725...,83.0,2.0,"[2672.953688172129, 1886.3428898369107]",[POLYGON Z ((104.7685957254023 20.692691437389...,"[0.002672953688172129, 0.0018863428898369106]",4.559297e+03
4,proj_vH52lGTvcjbCGKG4dINKsXJU,30024.0,CO,KardiaNuts was established in 2018 through the...,NaN,[{'geometry': {'coordinates': [[[-67.927780151...,95.0,1.0,[37617833.87699253],[POLYGON ((-67.92778015136719 6.11085625059348...,[37.617833876992535],3.761783e+07


### Checking Trees Planted Per MetresQ

In [86]:
# Understanding the number of trees planted in each sqmetre
df_filtered['tree_metresq']=df_filtered['countPlanted']/ df_filtered['total_area']

In [87]:
df_sorted = df_filtered.sort_values(by='tree_metresq', ascending=False)
df_sorted.tail(20)

,id,countPlanted,country,description,firstTreePlanted,sites,survivalRate,polygon_count,polygon_areas,polygons,polygon_areas_km,total_area,tree_metresq
101,proj_wqgGnda7Je2791h2214996tY,51.0,KE,The project is geared towards the regeneration...,NaN,[{'geometry': {'coordinates': [[[34.4257982586...,NaN,1.0,[25467126.997591965],[POLYGON ((34.425798258663065 0.83014240632913...,[25.467126997591965],2.546713e+07,2.002582e-06
216,proj_hPqUnDk4OGNh3FsCfBnUCSIW,41.0,TZ,Magombera Forest is a small but crucial locati...,NaN,[{'geometry': {'coordinates': [[[37.0205265947...,NaN,1.0,[26157456.02541927],[POLYGON Z ((37.02052659475081 -7.854226314807...,[26.15745602541927],2.615746e+07,1.567431e-06
222,proj_DwJhIfEbhogXRMAeHCdxouaE,3.0,ZM,Zambia is one of the countries in Africa with ...,NaN,[{'geometry': {'coordinates': [[[29.9418303803...,NaN,2.0,"[10612496.66687588, 8949940.40167891]",[POLYGON Z ((29.94183038033476 -9.797095906059...,"[10.612496666875881, 8.94994040167891]",1.956244e+07,1.533551e-07
202,proj_DlbJv5ybKOprfqt8rEMwMxN7,23.0,UG,Uganda is suffering from the rapid loss of its...,NaN,[{'geometry': {'coordinates': [[[32.6905691235...,NaN,6.0,"[524910648.9665009, 382899184.16734755, 529355...",[POLYGON ((32.69056912351297 0.281916943642741...,"[524.9106489665008, 382.8991841673475, 52.9355...",2.109859e+09,1.090120e-08
143,proj_iS4tktbXYmiwI2vq2iqWbViX,3.0,MW,A Tiny Paradise: A Fruit Tree Garden and safe ...,NaN,[{'geometry': {'coordinates': [[[33.4289195356...,100.0,1.0,[35563594818.56289],[POLYGON ((33.42891953566266 -9.72935041815760...,[35563.59481856289],3.556359e+10,8.435593e-11
255,proj_D15HAcTHhdj85bodkJ0f81Sf,0.0,AE,Via the Trees Matter initiative Goumbook creat...,2020-09-06 00:00:00,[{'geometry': {'coordinates': [[[55.4889880570...,85.0,3.0,"[1228.8931941321378, 793.0778804023954, 56162....",[POLYGON Z ((55.48898805704442 25.430526918229...,"[0.0012288931941321377, 0.0007930778804023954,...",5.818490e+04,0.000000e+00
38,proj_b0QCfioZWIMpoR0Xe0uI3Ncl,0.0,US,A study developed by American Forests and Texa...,2022-02-25 00:00:00,[{'geometry': {'coordinates': [[[-96.788907051...,NaN,1.0,[93025.68482938739],[POLYGON ((-96.78890705108643 32.9766100906889...,[0.09302568482938739],9.302568e+04,0.000000e+00
55,proj_NBfuMOvJa9a9QOTnobptG2sh,0.0,UA,WHY MAMA? \r\n\r\nThis is a call of the smalle...,2020-01-08 00:00:00,[{'geometry': {'coordinates': [[[30.0937006778...,92.0,1.0,[190661.44065477035],[POLYGON ((30.09370067786844 45.86220883185129...,[0.19066144065477036],1.906614e+05,0.000000e+00
46,proj_6PKQjUdikQOPG0MwjlLwCFiF,0.0,MX,This project is part of the Reforestation Camp...,NaN,[{'geometry': {'coordinates': [[[-92.790703181...,NaN,1.0,[2728435695.2477317],[POLYGON ((-92.79070318186784 16.7396471863412...,[2728.4356952477315],2.728436e+09,0.000000e+00
61,proj_5nEBZS0R4Zv7XLSVgC2jdlpK,0.0,IN,The âGreen Schoolsâ mission is born out of...,NaN,[{'geometry': {'coordinates': [[[75.1483665403...,NaN,3.0,"[322.78252901476884, 362.0572987157594, 571.66...",[POLYGON Z ((75.14836654038696 19.767173000296...,"[0.00032278252901476887, 0.0003620572987157593...",1.256507e+03,0.000000e+00


In [88]:
# Creating new column 'Total_areas_acres' to understand area in acres
df_sorted['Total_areas_acres'] = df_sorted['total_area'] * 0.000247105

df_sorted["trees_per_acre"]=df_sorted['countPlanted']/ df_sorted['Total_areas_acres']
df_sorted.head()

,id,countPlanted,country,description,firstTreePlanted,sites,survivalRate,polygon_count,polygon_areas,polygons,polygon_areas_km,total_area,tree_metresq,Total_areas_acres,trees_per_acre
188,proj_kcFeNrstIRmgN9FhHkEIfctV,1888983.0,ZM,The Luanshya district is Zambia's most densely...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",90.0,NaN,None,None,None,0.0,inf,0.0,inf
29,proj_mHS0F02qEFlJZULq7MMji6Ke,611867.0,NaN,Die Mutterbaumschule von BÃ¤ume fÃ¼r Menschen ...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",90.0,NaN,None,None,None,0.0,inf,0.0,inf
205,proj_26OP7JiC5mapekmJ4OahxzD8,38614.0,UG,ITF was formed in Kenya in 1922 when Richard S...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",85.0,NaN,None,None,None,0.0,inf,0.0,inf
193,proj_DbGBDHbePvTCUgndUcvrJodr,6058.0,CI,The region of the TANDA located in the EAST zo...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",89.0,NaN,None,None,None,0.0,inf,0.0,inf
48,proj_Cw1ZAjHXvjlC5Dfel2pWf4GX,3413.0,IT,AYUDH Europe is an international youth movemen...,NaN,"[{'geometry': None, 'type': 'Feature', 'proper...",90.0,NaN,None,None,None,0.0,inf,0.0,inf


### Removing Outliers

In [89]:

df_filtered_drp_out = df_sorted.dropna(subset=['polygon_count'])

df_filtered_drp_out["polygon_count"]


183    3.0
36     1.0
25     1.0
111    1.0
94     1.0
      ... 
134    3.0
136    5.0
197    2.0
220    1.0
258    1.0
Name: polygon_count, Length: 248, dtype: float64

In [90]:
df_filtered_drp_out["polygons"]

183    [POLYGON ((34.953337 0.412687, 34.953162 0.412...
36     [POLYGON Z ((88.85416108691702 22.209765723163...
25     [POLYGON Z ((37.47067529334751 5.2782711888472...
111    [POLYGON Z ((38.31446346746357 -4.785710152886...
94     [POLYGON Z ((-72.02315139238411 -45.5378169353...
                             ...                        
134    [POLYGON Z ((97.30532111487759 3.5990693933378...
136    [POLYGON ((31.44443739005326 1.67498919269821,...
197    [POLYGON ((-72.28019413974526 -13.293942743384...
220    [POLYGON ((97.6904296875 2.569938616535896, 96...
258    [POLYGON Z ((34.85792012084519 0.2842693405027...
Name: polygons, Length: 248, dtype: object

In [92]:

def extract_first_polygon(polygon_list):
    if isinstance(polygon_list, list) and polygon_list:

        return polygon_list[0]
    return None

df_filtered_drp_out['geometry'] = df_filtered_drp_out['polygons'].apply(extract_first_polygon)


C:\Users\Angela\AppData\Local\Temp\ipykernel_11348\3141931912.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_drp_out['geometry'] = df_filtered_drp_out['polygons'].apply(extract_first_polygon)


In [93]:
def is_valid_geometry(geometry):
    return geometry is not None and geometry.is_valid

#Checking if each row has valid geometry
df_filtered_drp_out = df_filtered_drp_out[df_filtered_drp_out['geometry'].apply(is_valid_geometry)]


In [94]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'id': 'project_id_reported',
    'countPlanted': 'trees_planted_reported',
    'description': 'description_reported',
    'firstTreePlanted': 'planting_date_reported',
    'polygons': 'geometry_reported',
    'survivalRate': 'survival_rate_reported',
    'total_area':'site_sqkm'
}


df_filtered_drp_out.rename(columns=columns_rename_mapping, inplace=True)

In [95]:
df_filtered_drp_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239 entries, 183 to 258
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   project_id_reported     239 non-null    object 
 1   trees_planted_reported  233 non-null    float64
 2   country                 238 non-null    object 
 3   description_reported    239 non-null    object 
 4   planting_date_reported  127 non-null    object 
 5   sites                   239 non-null    object 
 6   survival_rate_reported  143 non-null    float64
 7   polygon_count           239 non-null    float64
 8   polygon_areas           239 non-null    object 
 9   geometry_reported       239 non-null    object 
 10  polygon_areas_km        239 non-null    object 
 11  site_sqkm               239 non-null    float64
 12  tree_metresq            233 non-null    float64
 13  Total_areas_acres       239 non-null    float64
 14  trees_per_acre          233 non-null    float

In [96]:
# creating unique site id as we are only given project id

df_filtered_drp_out['site_id_reported'] = ['reforest_site_{}'.format(i) for i in range(1, len(df_filtered_drp_out) + 1)]

In [109]:

columns_to_keep = ['site_id_reported','project_id_reported','trees_planted_reported', 'country',"description_reported","planting_date_reported","geometry_reported","survival_rate_reported","site_sqkm"]

#
df= df_filtered_drp_out[columns_to_keep]


df.head()

,site_id_reported,project_id_reported,trees_planted_reported,country,description_reported,planting_date_reported,geometry_reported,survival_rate_reported,site_sqkm
183,reforest_site_1,proj_Ho7HXcYLYtkHX3wgA6gr3zIE,14049156.0,KE,Kadhune Friends of Earth Environment Preservat...,NaN,"[POLYGON ((34.953337 0.412687, 34.953162 0.412...",70.0,11506.736174
25,reforest_site_2,proj_9ShooMmxbB3thnsKZ1rSAXEw,7000578.0,ET,The Konso are a tribe in South-West Ethiopia. ...,NaN,[POLYGON Z ((37.47067529334751 5.2782711888472...,80.0,300363.215619
111,reforest_site_3,proj_ON5HC3IfetF3TO1EMWxjV6Uc,102172.0,TZ,Lushoto District is one of the eleven councils...,2019-11-05 00:00:00,[POLYGON Z ((38.31446346746357 -4.785710152886...,94.0,9215.237138
94,reforest_site_4,proj_lUjjbuZsvQgC1Qu4UMsl5hQ6,343483.0,CL,Reforest Patagonia is a non-profit organizatio...,NaN,[POLYGON Z ((-72.02315139238411 -45.5378169353...,75.0,66254.696073
214,reforest_site_5,proj_Dx21gX18qbwkDs402aOK3fxf,5838.0,BR,The Amazon Reforest Alliance works to recover ...,NaN,[POLYGON ((-73.02807749563135 -7.5199706462972...,NaN,1696.016252


In [110]:
df["description_reported"][0]

'This project will be implemented at Farm of the Future, a member of the Ecosystem Restoration Communities movement. \nThrough the establishment of agroforestry systems, this project will prove to local communitiesâ\x80\x99 members and local authorities that there are eco-friendly and economically viable alternatives to the monoculture plantation model. \nFirst, a pilot agroforestry system will be planted at Farm of the Future. About 2,000 trees will be planted on a plot of 1ha. This area will function as a demonstration site. \nAfter the establishment of this demonstration site, at least three local smallholder farmers will establish agroforestry systems on their land and become in turn examples for the broader community. \nThis project has also an important educational goal: each tree planted will directly finance an agroforestry training program that will educate and empower at least 120 high school students per year and will ensure the long-term sustainability of the project. \nThe

In [112]:
df["geometry_reported"]

183    [POLYGON ((34.953337 0.412687, 34.953162 0.412...
25     [POLYGON Z ((37.47067529334751 5.2782711888472...
111    [POLYGON Z ((38.31446346746357 -4.785710152886...
94     [POLYGON Z ((-72.02315139238411 -45.5378169353...
214    [POLYGON ((-73.02807749563135 -7.5199706462972...
                             ...                        
134    [POLYGON Z ((97.30532111487759 3.5990693933378...
136    [POLYGON ((31.44443739005326 1.67498919269821,...
197    [POLYGON ((-72.28019413974526 -13.293942743384...
220    [POLYGON ((97.6904296875 2.569938616535896, 96...
258    [POLYGON Z ((34.85792012084519 0.2842693405027...
Name: geometry_reported, Length: 239, dtype: object

In [115]:


class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        elif isinstance(obj, (list, dict)):
            return json.dumps(obj)
        return super().default(obj)


output_directory = 'C:/Users/Angela/Forest_Monitoring/input/Plant_for_planet_shapefiles/'


features = []


for index, row in df.iterrows():
    polygons = row['geometry_reported']
    if polygons is None:
        continue

    for polygon in polygons:
        # Converting row properties to dict and handle unsupported data types
        properties = row.drop('geometry_reported').to_dict()
        properties = {k: CustomEncoder().encode(v) if isinstance(v, (list, dict)) else v for k, v in properties.items()}
        
        feature = {
            'type': 'Feature',
            'geometry': mapping(polygon),
            'properties': properties,
        }
        features.append(feature)

gdf = gpd.GeoDataFrame.from_features(features)


gdf.columns = [col[:100] for col in gdf.columns]
gdf.info()
# Saving to shapefile or csv format allowed in earth engine upload
gdf.to_file(f"{output_directory}all_polygons.shp")
gdf.to_csv('plant_planet.csv', index=False)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2367 entries, 0 to 2366
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   geometry                2367 non-null   geometry
 1   site_id_reported        2367 non-null   object  
 2   project_id_reported     2367 non-null   object  
 3   trees_planted_reported  2352 non-null   float64 
 4   country                 2366 non-null   object  
 5   description_reported    2367 non-null   object  
 6   planting_date_reported  1925 non-null   object  
 7   survival_rate_reported  1665 non-null   float64 
 8   site_sqkm               2367 non-null   float64 
dtypes: float64(3), geometry(1), object(5)
memory usage: 166.6+ KB


C:\Users\Angela\AppData\Local\Temp\ipykernel_11348\1340446332.py:39: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f"{output_directory}all_polygons.shp")
c:\Users\Angela\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
c:\Users\Angela\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'site_id_reported' to 'site_id_re'
  ogr_write(
c:\Users\Angela\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'project_id_reported' to 'project_id'
  ogr_write(
c:\Users\Angela\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field nam